In [1]:
import numpy as np

In [2]:
def _embed(x_1d, order, delay):
        """
        Time-delay embedding.
        Parameters
        ----------
        x_1d : 1d-array, shape (n_times)
            Time series
        order : int
            Embedding dimension (order)
        delay : int
            Delay.
        Returns
        -------
        embedded : ndarray, shape (n_times - (order - 1) * delay, order)
            Embedded time-series.
        """
        N = x_1d.size
        Y = np.empty((order, N - (order - 1) * delay))
        for i in range(order):
            Y[i] = x_1d[i * delay:i * delay + Y.shape[1]]
        return Y.T
    
def permen(x, order, delay, normalize=False):
    pe = np.zeros((x.shape[0], x.shape[2]))
    hashmult = np.power(order, np.arange(order))
    
    for i in range(x.shape[0]):
        for k in range(x.shape[2]):
            sorted_idx = _embed(x[i, :, k], order, delay).argsort(kind='quicksort')
            
            # associate unique integer to each permutation
            hashval = np.sum(np.multiply(sorted_idx, hashmult), axis=1)
            
            # return counts
            _, c = np.unique(hashval, return_counts=True)
            
            p = c / c.sum()
            pe[i, k] = -np.multiply(p, np.log2(p)).sum()
    if normalize:
        pe = pe / np.log2(np.math.factorial(order))
    
    return pe

def permen1(x, order, delay, normalize=False):
    hashmult = np.power(order, np.arange(order))
    sorted_idx = _embed(x, order, delay).argsort(kind='quicksort')
            
    # associate unique integer to each permutation
    hashval = np.sum(np.multiply(sorted_idx, hashmult), axis=1)

    # return counts
    _, c = np.unique(hashval, return_counts=True)

    p = c / c.sum()
    pe = -np.multiply(p, np.log2(p)).sum()
    
    if normalize:
        pe = pe / np.log2(np.math.factorial(order))
    
    return pe

In [4]:
from perment import embed_sort as fembed
from perment import permutationentropy

In [6]:
np.random.seed(51)
x = np.random.rand(500, 150, 3)
xf = np.asfortranarray(x.transpose([1, 2, 0]))

order = 3
delay = 1
nrm = False

In [7]:
np.allclose(
    permen(x, order, delay, nrm), 
    permutationentropy(xf, order, delay, nrm).T
)

True

In [8]:
%timeit permutationentropy(xf, order, delay, nrm)

6.81 ms ± 2.91 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%timeit permen(x, order, delay, nrm)

78.8 ms ± 1.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
n = 500
order = 3
delay = 1

np.random.seed(5)
y = np.around(np.random.rand(3, n, 3), 2)
yf = np.asfortranarray(y.transpose([1, 2, 0]))
msi = n - (order - 1) * delay

%timeit _embed(y[0, :, 0], order, delay).argsort(kind='quicksort')
%timeit fembed(msi, yf[:, 0, 0], order, delay)

16.7 µs ± 21 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
3.79 µs ± 9.04 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [11]:
np.allclose(
    _embed(y[0, :, 0], order, delay).argsort(kind='quicksort'), 
    fembed(msi, yf[:, 0, 0], order, delay)
)

True

In [8]:
%timeit dpqsort(x.copy(), idx.copy())

694 ns ± 0.415 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [9]:
%timeit qsort(x.copy(), idx.copy())

800 ns ± 0.421 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [10]:
%timeit np.argsort(x)

1.54 µs ± 4.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
